In [1]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime, Float
import os

In [2]:
load_dotenv()

DB_PASS = os.getenv("DB_PASS")
DB_USER = os.getenv("DB_USER")
DB_HOST = os.getenv("DB_HOST")

engine = create_engine(f"mysql://{DB_USER}:{DB_PASS}@{DB_HOST}/options")
conn = engine.connect()

In [3]:
import pandas as pd

In [4]:
import yahoo_fin.stock_info as si

In [5]:
tickers = pd.read_csv('tickers_csv.csv')['TICKER']

In [6]:
from tqdm import tqdm
data_dict = {}

In [11]:
for ticker in tqdm(tickers):
    pull = si.get_data(ticker=ticker, start_date='8/1/2020', index_as_date=False)
    pull = pull.drop(columns=['open', 'high', 'low', 'close', 'volume'])
    pull['MA'] = pull['adjclose'].rolling(window=30).mean()
    pull['STD'] = pull['adjclose'].rolling(window=30).std()
    pull['Upper'] = pull['MA'] + (pull['STD'] * 2)
    pull['Lower'] = pull['MA'] - (pull['STD'] * 2)
    pull = pull.dropna()
    data_dict[ticker] = pull

100%|██████████| 193/193 [00:21<00:00,  8.92it/s]


In [12]:
data_dict['SHOP'].columns

Index(['date', 'adjclose', 'ticker', 'MA', 'STD', 'Upper', 'Lower'], dtype='object')

In [13]:
data_dict['SHOP']

,date,adjclose,ticker,MA,STD,Upper,Lower
29,2020-09-14,931.830017,SHOP,1019.849335,56.816210,1133.481756,906.216914
30,2020-09-15,929.390015,SHOP,1014.727334,57.839219,1130.405772,899.048895
31,2020-09-16,885.179993,SHOP,1008.233333,61.102716,1130.438765,886.027902
32,2020-09-17,870.760010,SHOP,1000.770333,63.797334,1128.365000,873.175666
33,2020-09-18,901.340027,SHOP,994.544000,64.091677,1122.727355,866.360646
...,...,...,...,...,...,...,...
106,2021-01-04,1092.400024,SHOP,1097.039669,78.606658,1254.252984,939.826354
107,2021-01-05,1118.739990,SHOP,1102.105334,74.722956,1251.551247,952.659422
108,2021-01-06,1087.310059,SHOP,1105.398669,71.658122,1248.714913,962.082426
109,2021-01-07,1159.530029,SHOP,1111.382670,68.240282,1247.863234,974.902106


In [33]:
for ticker in tqdm(tickers):
    data_dict[ticker].to_sql('band_data', conn,
                            if_exists='append', index=False,
                            dtype={
                                'date':DateTime,
                                'adjclose':Float,
                                'ticker':String(10),
                                'MA':Float,
                                'STD':Float,
                                'Upper':Float,
                                'Lower':Float
                            })

100%|██████████| 193/193 [00:18<00:00, 10.60it/s]


In [35]:
from sqlalchemy import MetaData, Table, Column, select, and_, or_

In [36]:
meta = MetaData()

In [40]:
bands = Table('band_data', meta)
stmt = select([bands]).where(bands.c.ticker == 'SHOP')
selected_data = pd.read_sql_query(stmt, conn)

In [41]:
selected_data

,date,adjclose,ticker,MA,STD,Upper,Lower
0,2020-09-14,931.83,SHOP,1019.850,56.8162,1133.48,906.217
1,2020-09-15,929.39,SHOP,1014.730,57.8392,1130.41,899.049
2,2020-09-16,885.18,SHOP,1008.230,61.1027,1130.44,886.028
3,2020-09-17,870.76,SHOP,1000.770,63.7973,1128.36,873.176
4,2020-09-18,901.34,SHOP,994.544,64.0917,1122.73,866.361
...,...,...,...,...,...,...,...
73,2020-12-28,1147.06,SHOP,1067.840,95.9438,1259.73,875.952
74,2020-12-29,1171.61,SHOP,1076.280,93.4434,1263.17,889.397
75,2020-12-30,1163.00,SHOP,1084.410,89.8443,1264.10,904.721
76,2020-12-31,1131.95,SHOP,1091.670,84.2272,1260.13,923.220


In [48]:
stmt = select([bands]).where(bands.c.date > '2020-12-30')
pd.read_sql_query(stmt, conn)

,date,adjclose,ticker,MA,STD,Upper,Lower
0,2020-12-31,1131.9500,SHOP,1091.67000,84.227200,1260.13000,923.22000
1,2021-01-04,1089.7300,SHOP,1096.95000,78.613600,1254.18000,939.72300
2,2020-12-31,337.3200,ZM,404.77600,31.515200,467.80600,341.74600
3,2021-01-04,359.5900,ZM,402.92800,32.503000,467.93400,337.92200
4,2020-12-31,49.1300,LYFT,45.45380,4.510000,54.47380,36.43380
...,...,...,...,...,...,...,...
379,2021-01-04,38.9200,GUSH,38.28690,3.302110,44.89120,31.68270
380,2020-12-31,2.1200,AMC,3.34000,0.762102,4.86420,1.81580
381,2021-01-04,2.0293,AMC,3.29898,0.798798,4.89657,1.70138
382,2020-12-31,30.0200,PLAY,26.67220,1.800930,30.27400,23.07030


In [49]:
stmt = select([bands]).where(and_(bands.c.date > '2020-12-30', bands.c.ticker == 'SHOP'))
pd.read_sql_query(stmt, conn)

,date,adjclose,ticker,MA,STD,Upper,Lower
0,2020-12-31,1131.95,SHOP,1091.67,84.2272,1260.13,923.220
1,2021-01-04,1089.73,SHOP,1096.95,78.6136,1254.18,939.723
